## Proposal

## Introduction

According to Deloitte Digital (2015), digital interactions are expected to influence 64 cents
of every dollar spent in retail stores by the end of 2015, meaning that social media is increasing its direct impact on companies' revenues. Combined that with the acceleration of social media use in the pandemic, it's key for companies to understand what will engage consumers the most. 

Therefore, we will compare average Lifetime Post Total Consumptions/Lifetime Post Total Impressions ratio between different types of content (Status, Photo, Link, Video) and find out if there is a statistical difference between them. The idea being we want to see how effective each type of content is when converting looks into engagement.

## Preliminary Results

In [4]:
data = read_csv("home/dataset_Facebook")
data

ERROR: Error in read_csv("home/dataset_Facebook"): could not find function "read_csv"
